In [ ]:
import pandas
import numpy as np

import strawb

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# Get a working file

Get file from DB. The MiniSpectrometer files are labeled with two different dataProductCodes:

- 'MSSD': in the SDAQ hdf5 format since ~04.2021. One file covers spectra up to a day.
- 'MSRD': previous to ~04.2021 for a single measurement. One file covers one spectra.

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler(file_name='Default')  # loads the db

In [ ]:
# mask by device
mask = db.dataframe['deviceCode'] == 'TUMPMTSPECTROMETER001'

mask &= db.dataframe['dataProductCode'] == 'MSSD'  # we need the MSSD

# and add mask by 'measurement_type'
#mask &= db.dataframe['synced']

db.dataframe[mask]  # plot the table

## In case none of the files is synced, download some files

In [ ]:
mask = db.dataframe['deviceCode'] == 'TUMPMTSPECTROMETER001'
mask &= db.dataframe['dataProductCode'] == 'MSSD'

## select 1day frame around one biolumi event
timestamp = np.datetime64('2021-09-24T00:00:00')
mask &= db.dataframe.dateFrom >= pandas.Timestamp(np.datetime64(timestamp, "h"), tz="UTC")
mask &= db.dataframe.dateFrom < pandas.Timestamp(np.datetime64(timestamp, "h"), tz="UTC") + np.timedelta64(23, "h")

### show selected file from the DB - here its 1 file only
db.dataframe[mask]

In [ ]:
### this will download the selected file
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        #save_db=True
    )
    db.save_db()

## Load the data

In [ ]:
mini_spec = strawb.sensors.MiniSpectrometer(file=db.dataframe.fullPath[mask].iloc[0])
mini_spec.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# In a module there can be multiple MiniSpectrometers.
# Therefore strawb.sensors.MiniSpectrometer imports them in a list called: mini_spectrometer_list.
# The PMTSpectrometer has only 1 MiniSpectrometer. Extract it here to a own variable for simple access.
mini_spec_1 = mini_spec.file_handler.mini_spectrometer_list[0]

In a module there can be multiple MiniSpectrometers.
Therefore `strawb.sensors.MiniSpectrometer` imports them in a list called: `mini_spectrometer_list`.
The PMTSpectrometer has only 1 MiniSpectrometer. Extract it here to a own variable for simple access.

In [ ]:
mini_spec_1 = mini_spec.file_handler.mini_spectrometer_list[0]

# Plots

## Average ADC value per pixel

In [ ]:
avg = np.average(mini_spec_1.adc_counts,axis=0)
scatter = go.Scatter(x=mini_spec_1.wavelength_arr,
                     y=np.average(mini_spec_1.adc_counts,axis=0), 
                     error_y=go.scatter.ErrorY(array=np.std(mini_spec_1.adc_counts,axis=0)), 
                     hoverlabel = dict(namelength = -1),
                     mode='markers',
                 )

fig = go.Figure(data=scatter)
fig.update_layout(
    xaxis_title=r"Wavelength [nm]",
    yaxis_title='Mean ADC +- Sigma [a.u.]',
    )

fig.show()

## Delta to average in sigma

In [ ]:
def plot_samples(mask):
    scatter_list = []
    index_arr = np.argwhere(mask).flatten()
    colors_arr = px.colors.sample_colorscale('viridis',index_arr.shape[0])

    for i, index_i in enumerate(index_arr):
        scatter = go.Scatter(x=mini_spec_1.wavelength_arr,
                             y=(mini_spec_1.adc_counts[index_i] - np.average(mini_spec_1.adc_counts,axis=0)) / np.std(mini_spec_1.adc_counts,axis=0), 
                             name=str(mini_spec_1.time.asdatetime()[index_i]).split('.',1)[0],
                             text=[str(mini_spec_1.time.asdatetime()[index_i]).split('.',1)[0]],
                             hoverlabel = dict(namelength = -1),
                             line_color = colors_arr[i],
                             mode='markers',
    #                          hovertemplate="x: %{x:.2f}<br>" +
    #                             "y: %{y:.2f} %{_xother}<br>"+
    #                             "text: %{text[0]} <br>,"
    #                             "<extra></extra>",
                         )
        scatter_list.append(scatter)

    fig = go.Figure(data=scatter_list)
    fig.update_layout(
        xaxis_title=r"Wavelength [nm]",
        yaxis_title='Delta ADC_i / Sigma_i ',
        )

    fig.show()
    

### For the period around a birth event in the camera and PMTs: 2021-09-24T21:57:27

In [ ]:
time_mask = mini_spec_1.time.asdatetime()[:] > np.datetime64('2021-09-24T21:45:00')
time_mask &= mini_spec_1.time.asdatetime()[:] < np.datetime64('2021-09-24T22:05:00')
    
plot_samples(time_mask)

### For a different period as reference

In [ ]:
time_mask = mini_spec_1.time.asdatetime()[:] > np.datetime64('2021-09-24T10:45:00')
time_mask &= mini_spec_1.time.asdatetime()[:] < np.datetime64('2021-09-24T11:05:00')
    
plot_samples(time_mask)